The standard way to avoid overfitting is called **L2 regularization**. It consists of appropriately modifying your cost function, from:
$$J = -\frac{1}{m} \sum\limits_{i = 1}^{m} \large{(}\small  y^{(i)}\log\left(a^{[L](i)}\right) + (1-y^{(i)})\log\left(1- a^{[L](i)}\right) \large{)}$$
To:
$$J_{regularized} = \small \underbrace{-\frac{1}{m} \sum\limits_{i = 1}^{m} \large{(}\small y^{(i)}\log\left(a^{[L](i)}\right) + (1-y^{(i)})\log\left(1- a^{[L](i)}\right) \large{)} }_\text{cross-entropy cost} + \underbrace{\frac{1}{m} \frac{\lambda}{2} \sum\limits_l\sum\limits_k\sum\limits_j W_{k,j}^{[l]2} }_\text{L2 regularization cost} $$

### These will be the changes in the cost and the backprop.
<br>
The cost function will be as follows:

In [ ]:
def compute_cost_with_regularization(AL, Y, parameters,regularization = "None",lambd = 0.7):
    m = Y.shape[1]
    # Extracting the weights from the dictionary
    i=0
    sum_of_W = 0
    for W in parameters.values():
        if i%2 == 0:
            sum_of_W += np.sum(np.square(W))
        i+=1
    m = Y.shape[1]
    cost = (1./m) * (-np.dot(Y,np.log(AL).T) - np.dot(1-Y, np.log(1-AL).T))
    
    cost = np.squeeze(cost)      # To make sure your cost's shape is what we expect (e.g. this turns [[17]] into 17).
    assert(cost.shape == ())
    if regularization == "None":
        lambd = 0
    L2_regularization_cost = lambd * sum_of_W / (2 * m)
    cost = cost + L2_regularization_cost
    
    return cost

And the backprop will be changed to the function given below

In [ ]:
def L_model_backward(Y_hat, Y, caches, parameters, regularization = "None",lambd = 0.7):
    m= Y.shape[1]
    L = len(caches)
    gradients = {}
    if regularization == "None":
        lambd = 0
    dA_prev = - (np.divide(Y, Y_hat) - np.divide(1 - Y, 1 - Y_hat));
    current_cache = caches[L-1]
    _,WL,_ = current_cache[0]
    gradients["dA" + str(L-1)], dW, gradients["db" + str(L)] = linear_activation_backward(dA_prev, current_cache, activation='sigmoid')
    gradients["dW" + str(L)] = dW + (lambd * WL / m)
    for l in reversed(range(L-1)):
        current_cache = caches[l]
        _,WL,_ = current_cache[0]
        dA_prev_temp, dW_temp, db_temp = linear_activation_backward(gradients["dA"+str(l+1)], current_cache, activation='relu')
        gradients["dA" + str(l)] = dA_prev_temp
        gradients["dW" + str(l + 1)] = dW_temp + (lambd * WL / m)
        gradients["db" + str(l + 1)] = db_temp
        
    return gradients

## The model therefore will call the new functions that implement the regularization.

In [ ]:
def L_layer_model(X, Y, layers_dims, learning_rate = 0.01, num_iterations = 3000, print_cost=False, lambd = 0.7):#lr was 0.009

    np.random.seed(1)
    costs = []                         # keep track of cost
    
    parameters = initialize_parameters_deep(layers_dims)
    
    # Loop (gradient descent)
    for i in range(0, num_iterations):

        # Forward propagation: [LINEAR -> RELU]*(L-1) -> LINEAR -> SIGMOID.
        AL, caches = L_model_forward(X, parameters)
        
        # Compute cost.
        cost = compute_cost_with_regularization(AL, Y, parameters, lambd = lambd, regularization = "L2" )
    
        # Backward propagation.
        grads = L_model_backward(AL, Y, caches, parameters, lambd )
 
        # Update parameters.
        parameters = update_parameters(parameters, grads, learning_rate)
                
        # Print the cost every 100 training example
        if print_cost and i % 100 == 0:
            print ("Cost after iteration %i: %f" %(i, cost))
        if print_cost and i % 100 == 0:
            costs.append(cost)
            
    # plot the cost
    plt.plot(np.squeeze(costs))
    plt.ylabel('cost')
    plt.xlabel('iterations (per hundreds)')
    plt.title("Learning rate =" + str(learning_rate))
    plt.show()
    
    return parameters